[View in Colaboratory](https://colab.research.google.com/github/freesemt/stewart-acid-base-model/blob/master/notebooks/StewartModel.ipynb)

In [1]:
%%writefile local_modules/OurUtils/StewartModel.py
"""
    StewartModel.py

    Copyright (c) 2018, Masatsuyo Takahashi
"""
import numpy as np

Ka  = 3e-7
Kw  = 4.4e-14
Kc  = 2.46e-11
K3  = 6e-11

MICRO   = 1e6
NANO    = 1e9

def Fh_impl(SID, Atot):
    p   = [ 1,
            Ka + SID,
            Ka*(SID - Atot) - Kw,
            -Ka*Kw
            ]
    x = sorted( list( np.roots( p ) ) )
    return x[-1]

def Fh(SID, Atot):
    assert np.isscalar(Atot)

    if np.isscalar(SID):
        return Fh_impl(SID, Atot)
    else:
        return np.array( [ Fh_impl(x, Atot) for x in SID ] )

def Fh4_impl(SID, Atot, pCO2):
    p   = [ 1,
            Ka + SID,
            Ka*(SID - Atot) - Kw - Kc*pCO2,
            -(Ka*(Kw + Kc*pCO2) - K3*Kc*pCO2),
            -Ka*K3*Kc*pCO2
            ]
    x = sorted( list( np.roots( p ) ) )
    # print( x )
    return x[-1]

def Fh4(SID, Atot, pCO2):
    # assert np.isscalar(SID) or np.isscalar(pCO2)
    assert np.isscalar(Atot)

    if np.isscalar(SID):
        if np.isscalar(pCO2):
            return Fh4_impl(SID, Atot, pCO2)
        else:
            return np.array( [ Fh4_impl(SID, Atot, x) for x in pCO2 ] )
    else:
        if np.isscalar(pCO2):
            return np.array( [ Fh4_impl(x, Atot, pCO2) for x in SID ] )
        else:
            assert len(SID.shape) == 2 and len(pCO2.shape) == 2 and SID.shape == pCO2.shape
            row_list = []
            for i in range(SID.shape[0]):
                row = []
                for j in range(SID.shape[1]):
                    s = SID[i,j]
                    p = pCO2[i,j]
                    h = Fh4_impl(s, Atot, p)
                    row.append(h)
                row_list.append(row)
            return np.array( row_list )


Writing local_modules/OurUtils/StewartModel.py
